# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

DemographicPerson (__id__, gender, age)\
MedicationsDrug (__code__, name)\
MedicationsUse (__personId__, __drugCode__, daysUse)\
-CHE: personId para DemographicPerson\
-CHE: drugCode para Medications

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

In [3]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [4]:
CREATE TABLE Medicamento (
    Code VARCHAR(6) NOT NULL,
    Name VARCHAR(80),
    PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

In [5]:
SELECT * FROM Medicamento;

In [6]:
CREATE TABLE UsoMedicamento (
    Person_Id VARCHAR(5) NOT NULL,
    Drug_Code VARCHAR(6) NOT NULL,
    Days_Use INTEGER,
    FOREIGN KEY(Person_Id)
        REFERENCES Pessoa(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Drug_Code)
        REFERENCES Medicamento(Code)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [7]:
SELECT * FROM UsoMedicamento;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [8]:
SELECT *
    FROM Pessoa P
    WHERE P.Age > 60;

b) Liste as idades das pessoas cadastradas (sem repetições)

In [9]:
SELECT DISTINCT P.Age
    FROM Pessoa P;

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [10]:
SELECT P.Id, P.Age, MU.Drug_Code, MU.Days_Use
    FROM Pessoa P, UsoMedicamento MU
    WHERE P.Id = MU.Person_Id;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [11]:
SELECT P.Id, P.Age, M.Name, MU.Days_Use
    FROM Pessoa P, Medicamento M, UsoMedicamento MU
    WHERE P.Id = MU.Person_Id AND M.Code = MU.Drug_Code
    ORDER BY P.Id, M.Name;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [12]:
SELECT DISTINCT M.Name
    FROM Medicamento M
    ORDER BY M.Name;

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [13]:
SELECT DISTINCT M.Name
    FROM Medicamento M, UsoMedicamento MU
    WHERE M.Code = MU.Drug_Code AND MU.Days_Use > 2000
    ORDER BY M.Name;